# Commissioning of beamline 5.3.1
awojdyla@lbl.gov, July 2024

Goals:
+ recenter the beam on the Be window
+ demonstrate 4 keV operations
+ attempt operation 
+ characterize drift in the system
+ test bluesky with the pilatus camera

What would be nice:
+ install x-ray fluorescence sensor with EPICS
+ add relays for camers switch and vacuum

In [1]:
import epics
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from time import sleep as time_sleep
from tqdm import tqdm

## Several useful EPICS controls

### Activate sample slits

In [99]:
epics.caput('DMC02:E.CNEN', 1)
epics.caput('DMC02:F.CNEN', 1)
epics.caput('DMC02:G.CNEN', 1)
epics.caput('DMC02:H.CNEN', 1)

1

In [113]:
epics.caget('DMC02:E.CNEN')
epics.caget('DMC02:F.CNEN')
epics.caget('DMC02:G.CNEN')
epics.caget('DMC02:H.CNEN')

1

In [121]:
slit_left_pos = epics.caget('DMC02:E')
slit_right_pos = epics.caget('DMC02:F')
slit_top_pos = epics.caget('DMC02:G')
slit_bot_pos = epics.caget('DMC02:H')

#    G-V
# E- →   <- F+  
#    H+^


print(slit_left_pos)
print(slit_right_pos)
print(slit_top_pos)
print(slit_bot_pos)

-84.0
64.0
-30.0
0.0


### Move sample slits

In [108]:
epics.caput('DMC02:E', -84)

1

In [75]:

epics.caput('DMC02:F', 64)

1

In [118]:
#epics.caput('DMC02:G', -35)
epics.caput('DMC02:G', -30)

1

In [123]:
epics.caput('DMC02:H', -50)

1

### deactivate sample slits

In [98]:
epics.caput('DMC02:E.CNEN', 0)
epics.caput('DMC02:F.CNEN', 0)
epics.caput('DMC02:G.CNEN', 0)
epics.caput('DMC02:H.CNEN', 0)

1

## Monochromator commissioning




In [124]:
# define mono angle to energy conversion

#physical constants
h_m2kgps = 6.6261e-34
c_mps    = 299792458   
e_eV     = 6.2415e+18
# Silicon spacing (for Bragg condition)
Si_m = 5.43E-10
a_Si111_m = Si_m/np.sqrt(1^2+1^2+1^2)
# Monochromotor offset
mono_offset_deg = 12.787
#Wavelength from energy
lambda_m = lambda E_eV : h_m2kgps*c_mps*e_eV/E_eV
#Bragg angle from Wavelength
bragg_deg = lambda E_eV: np.arcsin(lambda_m(E_eV)/(2*a_Si111_m))*180/np.pi
# Mono angle from Bragg angle
mono_deg = lambda E_eV: bragg_deg(E_eV)+mono_offset_deg

# inverse function to go from mono angle to eV
energy_eV = lambda mono_deg_input: (h_m2kgps*c_mps*e_eV)/((2 * a_Si111_m) * np.sin((np.pi/180)*(mono_deg_input - mono_offset_deg)))

## Pilatus interaction

In [4]:
#sample
series_nb = "%03.0f" % 0
image_nb = "%03.0f"  % 0
current_filename = "20240617_commissioning" + series_nb + "_" +image_nb
epics.caput('13PIL1:cam1:FileName',current_filename)
epics.caput('13PIL1:cam1:Acquire',1)

### bluesky!

In [182]:
epics.caput('bl531_xps1:mono_angle_deg',mono_deg(4000))

1